In [1]:
#pip install requests

Note: you may need to restart the kernel to use updated packages.


In [1]:
#pip install selenium

   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   -------------------------------- ------- 7.6/9.4 MB 42.7 MB/s eta 0:00:01
   ---------------------------------------- 9.4/9.4 MB 30.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time

driver_path = "chromedriver-win64/chromedriver" #크롬 드라이버 사용
service = Service(excutable_path = driver_path)
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

#review_url이라고 써있는 메모장에 있는 url을 하나씩 집어넣었습니다.
url = "https://m.brand.naver.com/stanley_korea/products/10118870118?nl-query=%ED%85%80%EB%B8%94%EB%9F%AC&nl-au=a97a8ea30d5f452f83bb1e17f30b0d65&NaPm=ci%3Da97a8ea30d5f452f83bb1e17f30b0d65%7Cct%3Dma7oljq7%7Ctr%3Dnslctg%7Csn%3D10106606%7Chk%3D445e72724e62a4eaedec4dfcc8000e8ce3f02f9d"  # 크롤링할 URL로 변경
driver.get(url)
time.sleep(3)
body = driver.find_element(By.TAG_NAME, "body")

x = 20
y = 7
# Page Down 키를 x번 누름
for _ in range(x):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.2)  # 각 키 입력 후 대기 시간

time.sleep(2)
reviews = []
rating = []
pagenum = 8
#킨토 빈플러스 코누는 지금 있는 코드로 그대로 사용하면 되고, 스탠리 락앤락 모슈는 range를 3에서 12로 바꾸고 while true 주석을 없앤 후 단락에 있는 부분의 주석을 풀어서 실행해야 됩니다.
#킨토 빈플러스 코누는 리뷰 개수를 20으로 나눠서 range의 마지막에 넣은거라서 개수와 조금 다를 수도 있습니다.
try:
#    while True:  # 계속해서 페이지를 반복
        for page in range(1, 126):  # a[3]부터 a[11]까지 순차적으로 클릭  
            try:
                
                # 해당 페이지에서 리뷰들 가져오기
                review_elements = driver.find_elements(By.CSS_SELECTOR, "span._2L3vDiadT9") #리뷰의 css 부분분
                for element in review_elements:
                    reviews.append(element.text)
                
                for i in range(1,21): #한 페이지에 리뷰가 20개 있으므로 20개의 별점을 가져옴 
                    rating_elements = driver.find_element(By.XPATH, '//*[@id="REVIEW"]/div/div[3]/div[2]/ul/li['+str(i)+']/div/div/div/div[1]/div/div[1]/div[1]/div[2]/div[1]/span/span')
                    style_value = rating_elements.get_attribute("style")
                    width_value = int(style_value.split(":")[1].strip().replace('%', '').replace(';', '').strip())
                    rating.append(width_value/20)

                # 페이지 번호 클릭 (a[3]부터 a[11]까지)
                #-----------------------------------
                # page_button = driver.find_element(By.CSS_SELECTOR, "#REVIEW > div > div._2LvIMaBiIO > div._2g7PKvqCKe > div > div > a:nth-child(" + str(page) + ")")
                # page_button.click()
                # time.sleep(2)  # 페이지 로딩 대기
                # for _ in range(6):
                #     body.send_keys(Keys.PAGE_DOWN)
                #     time.sleep(0.2)
                # time.sleep(2)
                #-----------------------------------
                next_button = driver.find_element(By.XPATH, '//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a['+ str(pagenum)+']')
                next_button.click()
                for _ in range(y):
                    body.send_keys(Keys.PAGE_DOWN)
                    time.sleep(0.2)
                time.sleep(2)  # '다음' 페이지로 이동 시간 기다리기
                if(pagenum<12) :
                    pagenum = pagenum+1;

            except Exception as e:
                print(f"페이지 {page} 버튼을 클릭할 수 없습니다. 더 이상 버튼이 없습니다.")
                break  # 버튼이 없으면 종료

        # '다음' 버튼 클릭 (a[12])
        #-----------------------------------
        # try:
        #     next_button = driver.find_element(By.XPATH, '//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[12]')
        #     next_button.click()
        #     for _ in range(y):
        #         body.send_keys(Keys.PAGE_DOWN)
        #         time.sleep(0.2)
        #     time.sleep(3)  # '다음' 페이지로 이동 시간 기다리기
        # except Exception:
        #     print("다음 버튼을 찾을 수 없습니다. 더 이상 페이지가 없습니다.")
        #     break  # '다음' 버튼이 없으면 종료
        #-----------------------------------

except Exception as e:
    print("오류 발생:", e)

finally:
    driver.quit()

페이지 3 버튼을 클릭할 수 없습니다. 더 이상 버튼이 없습니다.
다음 버튼을 찾을 수 없습니다. 더 이상 페이지가 없습니다.


In [4]:
#print(reviews)

['25.06.04.', '스텐리 텀블러 보냉력은 정말 좋아요\n퀜처 하나 사용중인데 추가로 구매했습니다\n차에 가지고 다니기에도 컵홀더에 딱 맞고\n빨대도 튼튼합니다\n색도 너무예쁘고 빨대 고정되는 실리콘까지.. 감동이예요', '25.06.02.', '커피 한번 사면 오래먹는 타입이라 얼음 빨리 안 녹는 텀블러가 필요했는데 기능이 너무 좋아요!! 사진은 커피 사고 4시간 지난 상태예요~ 색도 예쁘고 손잡이 없어도 밑에 둘레가 작아서 들고 다니기에 불편하지 않아요!! 강추합니다', '25.05.18.', '여자친구가 선물해 줬는데 너무 예뻐서 여자친구에게도 선물 했습니다 너무 예뻐요ㅎㅎ보냉? 스탠리인데 뭐 말할 필요가 있을까요ㅎㅎ 기능은 전세계적으로 인정을 받은곳이고 주문하시기전 색감들이 문제실텐데 최근들어 스탠리 3개 사본 결과 색은 고민하지 마세요 뭘 시키던 미친색감 옵니다ㅎㅎ 대만족~^^', '25.05.27.', '딸에게 선물했는데 실물이 훨씬 예뻐요.\n사진으로는 색상이 진하게 보이는데 실제 보면 화이트에 블루 한방울의 아주 연한 색이라 너무 예뻐요. 딸이 아주 맘에 들어하네요.', '25.05.19.', '기존에 887사용중이었는데 회사에선 좀 작은걸 원해서 591구입했어요.색상만 더 다양했으면 591로만 샀을 거 같아요.손잡이가 없지만 잡고쓰는데 전혀 불편함은 없어요.손이 작으시면 한손에 잡긴 좀 불편하실듯.부츠커버는 품절이라 타사이트에서 구입했어요.쨍한핑크라 페일스톤 끼었는데 잘 어울려요.페일스톤이 완전 백색이 아니라 크리미해서 더 이쁜듯해요~\n어차피 기능은 좋은거 알기에 만족해요🩷', '25.05.27.', '친구한테 선물했는데 넘 좋아해서 다른 색으로 또 선물 보냈어요. 가격도 괜찮고 만족도가 높아서 선물용으로도 좋은듯합니다!!', '25.05.28.', '여름에 없으면 안되는 필수템입니다\n\n피크닉이나 여행 시 차에 두고 먹기도 좋아요\n아이스크림, 아이스 커피 등 잘 사용하고 있어요\n다음날까지도 얼음 그대로 있어서 매우 만족합니다\n

In [9]:
import pandas as pd

#reviews의의 날짜와 리뷰를 분리
data = {
    "date" : reviews[0::2],
    "review" : reviews[1::2],
    "rating" : rating
}

df = pd.DataFrame(data)
df.to_csv('konu_reviews.csv',encoding="utf-8-sig")
#url딴 브랜드 명으로 바꾼 후 csv파일로 내보내기보내기